<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/bgRemoval/tests/PaddleSeg_matting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PaddleSeg - matting + BG-inpainting

[GitHub](https://github.com/PaddlePaddle/PaddleSeg/tree/release/2.5/Matting)


## 1. Preparations

Before start, make sure that you choose

Runtime Type = Python 3
Hardware Accelerator = GPU

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## 2. linking nextcloud

Connecting to the external NextCloud drive 

In [ ]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='bgRemoval',library='PaddleSeg')

## 3. install PaddlePaddle

In [4]:
!pip install -q PaddlePaddle

     |████████████████████████████████| 108.4 MB 25 kB/s 


## 4. clone GIT repo

In [5]:
import os
root_path = '/content/PaddleSeg'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/PaddlePaddle/PaddleSeg {root_path}

%ls

Cloning into '/content/PaddleSeg'...
remote: Enumerating objects: 17638, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 17638 (delta 53), reused 92 (delta 35), pack-reused 17518
Receiving objects: 100% (17638/17638), 341.87 MiB | 35.45 MiB/s, done.
Resolving deltas: 100% (11256/11256), done.
database/  database_mod.py  PaddleSeg/  __pycache__/  sample_data/


## 5. Setting up the environment

In [6]:
# installing PaddleSeg
%cd {root_path}
!pip install -q -r requirements.txt
!pip install -e .

# installing Matting
%cd Matting
!pip install -q -r requirements.txt

/content/PaddleSeg
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '-q'
Obtaining file:///content/PaddleSeg
     |████████████████████████████████| 596 kB 5.2 MB/s 
     |████████████████████████████████| 2.7 MB 42.9 MB/s 
     |████████████████████████████████| 197 kB 58.6 MB/s 
     |████████████████████████████████| 127 kB 4.0 MB/s 
     |████████████████████████████████| 2.1 MB 29.3 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 42 kB 674 kB/s 
     |████████████████████████████████| 69 kB 6.2 MB/s 
     |████████████████████████████████| 2.0 MB 58.6 MB/s 
     |████████████████████████████████| 98 kB 5.8 MB/s 
     |████████████████████████████████| 8.8 MB 39.7 MB/s 
     |████████████████████████████████| 461 kB 55.5 MB/s 
  Created wheel for bce-python-sdk: filename=bce_python_sdk-0.8.64-py3-none-any.whl size=202973 sha256=d99244eb0e27a66d0d011a92e6305ccea4fcb06b0bf9d3cfa9c9b70c31c6e29b
  Stored

## 6. downloading the dataset

In [14]:
# download model checkpoint 
model_path = root_path + '/Matting/data/model'
model_params = 'https://paddleseg.bj.bcebos.com/matting/models/human_matting-resnet34_vd.pdparams'
model_inf = 'https://paddleseg.bj.bcebos.com/matting/models/deploy/pp-humanmatting-resnet34_vd.zip'
# make folders
os.makedirs(model_path, exist_ok=True)
if not os.path.exists(os.path.join(model_path,'human_matting.pdparams')):
  print('\n> Download the model params')
  !curl {model_params} -o {os.path.join(model_path,'human_matting.pdparams')}
else:
  print ('\n> File already downloaded')
if not os.path.exists(os.path.join(model_path,'human_matting-resnet.zip')):
  print('\n> Download the model')
  !curl {model_inf} -o {os.path.join(model_path,'human_matting-resnet.zip')}
  !unzip -q {os.path.join(model_path,'human_matting-resnet.zip')} -d {os.path.join(model_path)}
else:
  print ('\n> File already downloaded')


> File already downloaded

> File already downloaded


## 7. Deploy

In [ ]:
# infer
# !python deploy/python/infer.py \
#     --config output/export/deploy.yaml \
#     --image_path data/PPM-100/val/fg/ \
#     --save_dir output/results \
#     --fg_estimate True

mode = 'predict' #@param ['predict','bgremoval']
if(mode=='predict'):
# predict
  !export CUDA_VISIBLE_DEVICES=0
  !python predict.py \
      --config configs/human_matting/human_matting-resnet34_vd.yml \
      --model_path data/model/human_matting.pdparams \
      --image_path {input_folder} \
      --save_dir {output_folder} \
      --fg_estimate True
else:
  # bg replacement
  background = 'r' #@param ['r','g','b','w'] {allow-input: true}
  !export CUDA_VISIBLE_DEVICES=0
  !python bg_replace.py \
      --config configs/human_matting/human_matting-resnet34_vd.yml \
      --model_path data/model/human_matting.pdparams \
      --image_path {input_folder} \
      --background path/to/your/background/image \
      --save_dir {output_folder} \
      --fg_estimate True

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati